In [1]:
import numpy as np
import pandas as pd

from geopy.distance import distance
import datetime
import time

### Введение

#### Загрузите csv файл в датафрейм

In [2]:
citybike = pd.read_csv('201809-citibike-tripdata.csv')

In [4]:
citybike.head()


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1635,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,132,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,3337,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,436,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,8457,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


1. Найти общее количество строк и столбцов в датасете - указать первым число строк, вторым - число столбцов

In [12]:
citybike.shape

(1877884, 15)

Найти среднюю длину поездок в минутах(столбец tripduration) c точностью до 2 знака

In [3]:
citybike['tripduration'] = citybike['tripduration']/60
round(citybike.tripduration.mean(),2)
#round(citybike.describe()['tripduration']['mean'], 2)

16.13

Сколько поездок начались и закончились в той же самой станции?

In [5]:
citybike[(citybike['start station id'] == citybike['end station id'])].shape

(41364, 15)

Сколько поездок начались и закончились в той же самой станции? Указать количество уникальных байков

In [43]:
citybike[(citybike['start station id'] == citybike['end station id'])]['bikeid'].unique().shape

(9939,)

Какой самый используемый байк(bikeid) в городе по количеству поездок? 

In [7]:
citybike['bikeid'].value_counts()[:1]

33875    559
Name: bikeid, dtype: int64

Найдите байк(bikeid), у которого в среднем продолжительность поездок выше, чем у всех остальных

In [15]:
citybike.groupby('bikeid')['tripduration'].mean().sort_values(ascending=False)[:1]

bikeid
17548    8133.706667
Name: tripduration, dtype: float64

Найдите количество строк, в которых отсутствуют данные о start station id

In [17]:
citybike[citybike['start station id'].isnull()].shape

(716, 15)

Какова средняя продолжительность поездки в зависимости от типа подписки c точностью до 2 знака? 

In [18]:
round(citybike.groupby('usertype')['tripduration'].mean(),2)

usertype
Customer      33.42
Subscriber    13.33
Name: tripduration, dtype: float64

Для каждой станции найдите расстояние между станциями, а затем найдите среднее расстояние по всем поездкам, предварительно выкинув замкнутые траектории(те у которых совпадают start station id = end station id). 

Hint: можно воспользоваться библиотекой geopy и взять расстояние vincenty(минимальное расстояние между точками)


In [6]:
citybike['distance_km'] = citybike.apply(lambda x: distance((x['start station latitude'], x['start station longitude']),
                                                            (x['end station latitude'], x['end station longitude'])).kilometers, axis=1)

print(citybike['distance_km'].mean())



1.808845403072724


In [15]:
citybike['end station name'].value_counts()[:5]

Grove St PATH     2563
Exchange Place    1128
Hamilton Park     1074
Sip Ave            814
Newport PATH       721
Name: end station name, dtype: int64

Выберите станцию (start station id) с максимальным количеством отправлений с 18 до 20 вечера

In [24]:
citybike['end_hour'] = citybike['stoptime'].apply(lambda x: datetime.datetime.fromtimestamp(
            time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)

In [25]:
citybike['start_hour'] = citybike['starttime'].apply(lambda x: datetime.datetime.fromtimestamp(
                                        time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)

In [33]:
citybike[(citybike.start_hour.isin([18, 19, 20]))]['start station id'].value_counts().head()

519.0    3181
426.0    2701
402.0    2680
459.0    2605
477.0    2418
Name: start station id, dtype: int64

Выберите станции(end station id), в которые приезжают с 6 до 10 утра


In [29]:
end_stations = [3140, 3106, 3116, 369]

In [30]:
citybike[(citybike.end_hour.isin([6,7,8,9,10])) &
         (citybike['end station id'].isin(end_stations))]['end station id'].unique()

array([3106., 3116., 3140.,  369.])